<a href="https://colab.research.google.com/github/YanuarN/HarvestHeroes/blob/Iqbal/Machine%20Learning/Paddy_Disease_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-cc2d2d63-4436-2a56-e530-62b1d3d68b06)


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

2024-06-10 04:15:51.422042: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 04:15:51.422164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 04:15:51.539716: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
TRAIN_CSV = "/kaggle/input/paddy-disease-classification/train.csv"
train_df = pd.read_csv(TRAIN_CSV)
train_df.head()

,image_id,label,variety,age
0,100330.jpg,bacterial_leaf_blight,ADT45,45
1,100365.jpg,bacterial_leaf_blight,ADT45,45
2,100382.jpg,bacterial_leaf_blight,ADT45,45
3,100632.jpg,bacterial_leaf_blight,ADT45,45
4,101918.jpg,bacterial_leaf_blight,ADT45,45


In [ ]:
# Check distribution of labels in training data
train_df.label.value_counts()

label
normal                      1764
blast                       1738
hispa                       1594
dead_heart                  1442
tungro                      1088
brown_spot                   965
downy_mildew                 620
bacterial_leaf_blight        479
bacterial_leaf_streak        380
bacterial_panicle_blight     337
Name: count, dtype: int64

In [ ]:
TRAINING_DIR = '/kaggle/input/paddy-disease-classification/train_images'

# Data augmentation for training data
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1
)

# Data generator for the training data
train_generator = train_datagen.flow_from_directory(
    directory=TRAINING_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

# Get label class name
train_label = train_generator.class_indices
print(train_label)

Found 9371 images belonging to 10 classes.
{'bacterial_leaf_blight': 0, 'bacterial_leaf_streak': 1, 'bacterial_panicle_blight': 2, 'blast': 3, 'brown_spot': 4, 'dead_heart': 5, 'downy_mildew': 6, 'hispa': 7, 'normal': 8, 'tungro': 9}


In [ ]:
# Data generator for the validation data
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.1)
val_generator = validation_datagen.flow_from_directory(
    directory=TRAINING_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    subset='validation'
)

val_label = val_generator.class_indices
print(val_label)

Found 1036 images belonging to 10 classes.
{'bacterial_leaf_blight': 0, 'bacterial_leaf_streak': 1, 'bacterial_panicle_blight': 2, 'blast': 3, 'brown_spot': 4, 'dead_heart': 5, 'downy_mildew': 6, 'hispa': 7, 'normal': 8, 'tungro': 9}


In [ ]:
# Callback for our train model
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (logs.get("accuracy") > 0.94) and (logs.get("val_accuracy") > 0.94):
            print("\nMinimum accuracy reached, stop training")
            self.model.stop_training = True

callback = myCallback()

In [ ]:
# Load the pre-trained EfficientNetB4 model without the top classification layer
efficientnet_base = tf.keras.applications.EfficientNetB4(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained base model layers
efficientnet_base.trainable = False

71686520/71686520 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Build the model
model = tf.keras.models.Sequential()

# Add the pre-trained Xception base
model.add(efficientnet_base)

# Add global average pooling layer to reduce spatial dimensions
model.add(tf.keras.layers.AveragePooling2D((3,3)))

# Flatten the feature maps
model.add(tf.keras.layers.Flatten())

# Add a dense layer with 120 units and ReLU activation function
model.add(tf.keras.layers.Dense(220, activation='relu'))

# Dropout Layer
model.add(tf.keras.layers.Dropout(0.25))

# Add the output layer with 1 unit and sigmoid activation function for binary classification
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb4 (Functional)     │ ?                      │    17,673,823 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ ?                      │   0 (unbuilt) │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,673,823 (67.42 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 17,673,823 (67.42 MB)

In [ ]:
learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
 loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history= model.fit(train_generator,
          validation_data=val_generator,
          epochs=100,
          callbacks=[callback])

Epoch 1/100


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


FailedPreconditionError: Graph execution error:

Detected at node sequential_2_1/dense_1_2/Add/ReadVariableOp defined at (most recent call last):
<stack traces unavailable>
Read variable failure sequential_2/dense_1_1/bias/645. It could mean the variable is uninitialized or the variable is on another device 
	 [[{{node sequential_2_1/dense_1_2/Add/ReadVariableOp}}]]
	tf2xla conversion failed while converting __inference_one_step_on_data_36136[]. Run with TF_DUMP_GRAPH_PREFIX=/path/to/dump/dir and --vmodule=xla_compiler=2 to obtain a dump of the compiled functions.
	 [[StatefulPartitionedCall]] [Op:__inference_one_step_on_iterator_37449]

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_generator = train_generator.cache().prefetch(buffer_size=AUTOTUNE)
val_generator = val_generator.cache().prefetch(buffer_size=AUTOTUNE)

AttributeError: 'DirectoryIterator' object has no attribute 'cache'

In [ ]:
model=tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation="relu", input_shape=(224, 224, 3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(train_generator,
          epochs=50,
          validation_data=val_generator,
          callbacks=callback)

Epoch 1/50
292/293 ━━━━━━━━━━━━━━━━━━━━ 0s 532ms/step - accuracy: 0.2513 - loss: 2.5036

2024-06-10 04:32:00.605460: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 98573: 7.23958, expected 6.41177
2024-06-10 04:32:00.605517: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 98575: 6.82729, expected 5.99948
2024-06-10 04:32:00.605526: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 98576: 7.15024, expected 6.32244
2024-06-10 04:32:00.605534: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 98577: 6.7427, expected 5.9149
2024-06-10 04:32:00.605542: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 98579: 6.39704, expected 5.56924
2024-06-10 04:32:00.605550: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 98580: 6.60919, expected 5.78139
2024-06-10 04:32:00.605558: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 98581: 6.32347, expected 5.49567
2024-06-10 04:32:00.605565: E

293/293 ━━━━━━━━━━━━━━━━━━━━ 176s 574ms/step - accuracy: 0.2519 - loss: 2.5001 - val_accuracy: 0.4363 - val_loss: 1.7321
Epoch 2/50
293/293 ━━━━━━━━━━━━━━━━━━━━ 122s 406ms/step - accuracy: 0.4575 - loss: 1.5980 - val_accuracy: 0.5270 - val_loss: 1.3727
Epoch 3/50
293/293 ━━━━━━━━━━━━━━━━━━━━ 119s 397ms/step - accuracy: 0.5407 - loss: 1.3303 - val_accuracy: 0.6129 - val_loss: 1.1838
Epoch 4/50
293/293 ━━━━━━━━━━━━━━━━━━━━ 124s 412ms/step - accuracy: 0.6152 - loss: 1.1363 - val_accuracy: 0.6641 - val_loss: 0.9895
Epoch 5/50
293/293 ━━━━━━━━━━━━━━━━━━━━ 127s 422ms/step - accuracy: 0.6699 - loss: 1.0016 - val_accuracy: 0.6786 - val_loss: 0.9687
Epoch 6/50
293/293 ━━━━━━━━━━━━━━━━━━━━ 123s 409ms/step - accuracy: 0.7156 - loss: 0.8452 - val_accuracy: 0.7133 - val_loss: 0.9452
Epoch 7/50
293/293 ━━━━━━━━━━━━━━━━━━━━ 123s 409ms/step - accuracy: 0.7524 - loss: 0.7367 - val_accuracy: 0.7481 - val_loss: 0.8313
Epoch 8/50
293/293 ━━━━━━━━━━━━━━━━━━━━ 121s 403ms/step - accuracy: 0.7836 - loss: 0.65

In [ ]:
# Save model to h5 format
model.save("model_Padi_2.h5")